<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/main/CommonPythonMistakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common Python Mistakes

This notebook goes through a handful of Python pitfalls that come up again and again, especially early on. None of these are hard to fix once you know what to look for — the tricky part is that Python often doesn't throw an error, it just silently does something different from what you intended.

All examples use physics context, because that's what we're here for.

## 1. Defining a function is not the same as running it

This is probably the most common source of 'my code does nothing' confusion. When you write `def`, Python reads and stores the function — it does not execute it. You have to explicitly call the function afterwards to get any output.

Think of it like writing an equation on the board. Writing $F = ma$ doesn't calculate anything. You have to plug in numbers.

In [1]:
# This defines the function but produces no output
def kinetic_energy(mass, velocity):
    """Return the non-relativistic kinetic energy in Joules."""
    return 0.5 * mass * velocity**2

# Nothing happens yet — Python just knows the function exists

In [2]:
# You have to call it to get a result
KE = kinetic_energy(0.511e6, 0.5)   # electron at half the speed of light (eV units)
print(f"Kinetic energy: {KE:.3e} eV·(c/2)²")

# Or call it directly in a print
print("KE (1 kg at 10 m/s):", kinetic_energy(1, 10), "J")

Kinetic energy: 6.388e+04 eV·(c/2)²
KE (1 kg at 10 m/s): 50.0 J


**Rule of thumb:** after every `def` block, ask yourself — did I actually call this function? If there's no output in your notebook, the answer is probably no.

## 2. `/` vs `//` — division vs integer division

Python has two division operators and they behave very differently:

- `/` — regular division, always returns a float
- `//` — floor division, rounds **down** / truncates to the nearest integer

Using `//` when you mean `/` gives you a wrong numerical answer with no error message. This is especially dangerous in median calculations or anywhere you need a precise average.

In [15]:
# Example: averaging two measurements
a = 9.81   # gravitational acceleration at site A (m/s²)
b = 9.83   # gravitational acceleration at site B (m/s²)

print("Correct average:       ", (a + b) / 2)    # → 9.82  ✓
print("Integer division:      ", (a + b) // 2)   # → 9.0   ✗  (truncates!)

Correct average:        9.82
Integer division:       9.0


In [4]:
# Where this really hurts: median of an even-length list
measurements = [9.78, 9.80, 9.82, 9.85]   # four g measurements
measurements.sort()
n = len(measurements)

# The two middle values
mid_low  = measurements[n // 2 - 1]   # 9.80
mid_high = measurements[n // 2]       # 9.82

print("Middle values:", mid_low, mid_high)
print("Median with / :", (mid_low + mid_high) / 2)    # → 9.81  ✓
print("Median with //:", (mid_low + mid_high) // 2)   # → 9.0   ✗

Middle values: 9.8 9.82
Median with / : 9.81
Median with //: 9.0


`//` does have legitimate uses — for example, finding the middle index of a list (`n // 2`) where you *need* a whole number. The key is knowing which situation you're in:

- Index into a list → use `//`
- Compute a value → use `/`

## 3. Sets `{}` vs Dictionaries `{key: value}`

Both use curly braces, but they are completely different data structures. A **set** is an unordered collection of unique values. A **dictionary** maps keys to values and lets you look things up.

Python decides which one you get based on whether you include colons.

In [5]:
# This looks like a dictionary — it is NOT
particles_set = {"electron", "proton", "neutron"}
print(type(particles_set))   # → <class 'set'>
print(particles_set)          # unordered, no keys

# You cannot do this with a set:
# print(particles_set["electron"])   ← TypeError

<class 'set'>
{'proton', 'electron', 'neutron'}


In [19]:
p = ("e","p","n")
type(p)

tuple

In [6]:
# This IS a dictionary — note the key: value pairs
particles_dict = {
    "electron": 0.511,    # rest mass in MeV/c²
    "proton":   938.3,
    "neutron":  939.6
}
print(type(particles_dict))            # → <class 'dict'>
print(particles_dict["electron"])      # → 0.511  ✓
print(particles_dict["proton"])        # → 938.3  ✓

<class 'dict'>
0.511
938.3


In [7]:
# One more: you can loop over a dictionary cleanly
for particle, mass in particles_dict.items():
    print(f"{particle:10s}  {mass:.1f} MeV/c²")

electron    0.5 MeV/c²
proton      938.3 MeV/c²
neutron     939.6 MeV/c²


**Quick check:** if you need to look something up by name, you want a dictionary. If you just need to store a collection of unique items with no labels, a set is fine — but that's a much rarer situation.

In [20]:
# Example of a Tuple
my_tuple = ("apple", "banana", "cherry", "apple")
print(f"Tuple: {my_tuple}")
print(f"Type of my_tuple: {type(my_tuple)}")
print(f"Accessing an element by index (my_tuple[0]): {my_tuple[0]}")
print(f"Tuple allows duplicates: 'apple' appears {my_tuple.count('apple')} times.")

# my_tuple[0] = "orange"  # This would raise a TypeError because tuples are immutable

print("\n" * 2) # Add some spacing for readability

# Example of a Set
my_set = {"apple", "banana", "cherry", "apple"}
print(f"Set: {my_set}") # Note the order might not be the same as insertion and duplicates are removed
print(f"Type of my_set: {type(my_set)}")

# print(my_set[0]) # This would raise a TypeError because sets do not support indexing

my_set.add("orange")
print(f"Set after adding 'orange': {my_set}")
my_set.remove("banana")
print(f"Set after removing 'banana': {my_set}")

Tuple: ('apple', 'banana', 'cherry', 'apple')
Type of my_tuple: <class 'tuple'>
Accessing an element by index (my_tuple[0]): apple
Tuple allows duplicates: 'apple' appears 2 times.



Set: {'banana', 'cherry', 'apple'}
Type of my_set: <class 'set'>
Set after adding 'orange': {'orange', 'banana', 'cherry', 'apple'}
Set after removing 'banana': {'orange', 'cherry', 'apple'}


## 4. Calling a method vs referencing it — don't forget `()`

In Python, a method is just a function attached to an object. Writing `object.method` gives you the function itself. Writing `object.method()` actually calls it and gives you the result.

This one is easy to miss because Python doesn't throw an error — it just prints something like `<built-in method ...>` and moves on.

In [8]:
element = "hydrogen"

# Missing () — you get the method object, not the result
print(element.upper)     # → <built-in method upper of str object at ...>

# With () — you get the actual result
print(element.upper())   # → HYDROGEN  ✓

<built-in method upper of str object at 0x7af915145cf0>
HYDROGEN


In [9]:
# This applies to any method, not just strings
import math

angle_deg = 45
angle_rad = math.radians(angle_deg)   # radians() needs () to run

print("sin(45°):", math.sin(angle_rad))   # → 0.7071...

# Compare — no () on sin:
print(math.sin)   # → <built-in function sin>  — useless

sin(45°): 0.7071067811865475
<built-in function sin>


**Rule:** if you see `<built-in method ...>` or `<function ... at 0x...>` in your output, you forgot the parentheses.

## Bonus: Print indentation matters

This is a smaller one but it trips people up in `if/else` blocks. If your `print` is inside the `if` or `else`, it only runs in that branch. Put it outside if you always want it to print.

In [10]:
values = [1.2, 3.4, 5.6, 7.8]   # even-length list
n = len(values)
values.sort()
mid = n // 2

if n % 2 == 1:
    median = values[mid]
else:
    median = (values[mid - 1] + values[mid]) / 2
    print("(even-length list)")   # this only prints in the else branch

print("Median:", median)   # this always prints — put it here

(even-length list)
Median: 4.5
